<a href="https://colab.research.google.com/github/EduardoSchiavo/TensorFlowLab/blob/main/Exercises/Exercise%204%20-%20Handling%20Complex%20Images/Exercise%204-Question.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('accuracy')>DESIRED_ACCURACY:
      print('\n\n Desired Accuracy Reached. Stopping Training! \n\n')
      self.model.stop_training=True
  # Your Code

callbacks = myCallback()

--2022-01-20 14:29:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.128, 2607:f8b0:4023:c0b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.06s   

2022-01-20 14:29:54 (45.4 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', 
                                input_shape=(150,150, 3)),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'), 
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
])

# model.summary()

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [4]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
           '/tmp/h-or-s/',
           target_size=(150,150),
           batch_size=10,
           class_mode='binary')
          

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [5]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(train_generator, epochs=20, callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/20
8/8 [==============================] - 3s 263ms/step - loss: 2.1357 - accuracy: 0.4875
Epoch 2/20
8/8 [==============================] - 2s 256ms/step - loss: 0.5529 - accuracy: 0.7000
Epoch 3/20
8/8 [==============================] - 2s 260ms/step - loss: 0.1803 - accuracy: 0.9625
Epoch 4/20
8/8 [==============================] - 2s 257ms/step - loss: 0.3915 - accuracy: 0.8000
Epoch 5/20
8/8 [==============================] - 2s 260ms/step - loss: 0.1187 - accuracy: 0.9625
Epoch 6/20
8/8 [==============================] - 2s 256ms/step - loss: 0.2112 - accuracy: 0.9125
Epoch 7/20
8/8 [==============================] - 2s 260ms/step - loss: 0.0709 - accuracy: 0.9875
Epoch 8/20
8/8 [==============================] - 2s 259ms/step - loss: 0.0712 - accuracy: 0.9500
Epoch 9/20
8/8 [==============================] - ETA: 0s - loss: 0.0171 - accuracy: 1.0000

 Desired Accuracy Reached. Stopping Training! 


8/8 [==============================] - 2s 256ms/step - loss: 0.0171 - accu

Testing the model on real images

In [7]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = x / 255
  x = np.expand_dims(x, axis=0)
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " happy")
  else:
    print(fn + " sad")

Saving smiley.jpg to smiley.jpg
[0.9895066]
smiley.jpg happy
